In [1]:
import numpy as np
#import matplotlib.pyplot as plt
import numba as nb
#import sys
import openseespy.opensees as ospy

In [2]:
# versão do openseespy
ospy.version()

'3.0.3'

In [3]:
# target do numba
#targNumba = 'cuda'
targNumba = 'parallel'

In [4]:

@nb.vectorize(["float64(int64,int64,int64,float64)"], target= targNumba)
def coordX(M, vx, vy, dx):
    if M <  ( (vx+1) * (vy+1) ):
        return  (M % (vx+1) ) * (dx)
    else:
        return  ( ( M - (vx+1) * (vy+1) ) % (vx*2 +1) ) * (dx/2) 

In [5]:

@nb.vectorize(["float64(int64,int64,int64,float64)"], target= targNumba)
def coordY(M, vx, vy, dy):
    if M <  ( (vx+1)* (vy+1) ):
        return  ((M % ( (vx+1) * (vy+1) )// (vx+1) ) ) * (dy)
    else:
        return  ( ((M - (vx+1) * (vy+1) ) % ( (vx*2 +1) * (vy*2 +1) ))// (vx*2 +1) ) * (dy/2)

In [6]:

@nb.vectorize(["float64(int64,int64,int64,float64)"], target= targNumba)
def coordZ(M, vx, vy, dz):
    if M < ( (vx+1)* (vy+1) ):
        return  0.0
    else:
        return  (((M - ( (vx+1)* (vy+1) ) ) // ( (vx*2 +1)*(vy*2 +1))) + 1) * (dz)

In [7]:

@nb.jit(["void(int64,float64,float64,float64)"], parallel = True)
def ospyNode(M, cx, cy, cz):
    for i in nb.prange(M.size):
        ospy.node(int(M[i]),cx[i],cy[i],cz[i])
        if cz[i] == 0.0:
            ospy.fix(int(M[i]), 1, 1, 1, 1, 1, 1) # border condition

## Função vectorize não fnciona

@nb.vectorize(["void(float64, float64, float64, float64, float64, float64, int64, int64, int64, int64)"],  target= 'parallel')
def createElement(A, E, G, J, Iy, Iz,  gTTag,  matINd, matFNd, matTags ):
        ospy.element('elasticBeamColumn', int(matTags) , int(matINd), int(matFNd), A, E, G, J, Iy, Iz, gTTag ,  '-mass', 0.0,  '-cMass')


@nb.jit(["void(float64, float64, float64, float64, float64, float64, int64, int64, int64, int64)"], parallel = True)
def createElement(A, E, G, J, Iy, Iz,  gTTag,  matINd, matFNd, matTags ):
    for i in nb.prange( matFNd.size):
        ospy.element('elasticBeamColumn', int(matTags[i]) , int(matINd[i]), int(matFNd[i]), A, E, G, J, Iy, Iz, gTTag ,  '-mass', 0.0,  '-cMass')
    

In [8]:

@nb.jit(["void(float64,float64,float64, float64, int64, int64, int64, int64)"], parallel = False)
def frameSet(h, b, fck, ni,  gTTag,  matINd, matFNd, startTag):
    A = b*h # area do elemento
    E = .85*5600*(fck**.5)*(10**3) # modulo de elasticidade - Young modulos (transformando de Mpa para KN/m2 -> 10**3)
    G = E/(2*(1+ni)) # modulo de cisalhamento - shear modulus
    J = ((b*h)/12) * (b**2 + h**2) # torsional moment of inertia of cross section
    Iy = (b*h**3)/12 # second moment of area about the local z-axis
    Iz = (h*b**3)/12 # second moment of area about the local y-axis
    for i in nb.prange(matINd.size):
        ospy.element('elasticBeamColumn', i + startTag, int(matINd[i]), int(matFNd[i]), A, E, G, J, Iy, Iz, gTTag , '-mass', 0.0,'-cMass')

In [9]:

@nb.jit(["void()"])
def resetModel():
    # - Modelo Openseespy
    # erase the model
    ospy.wipe()

    # - init model
    # degrees of freedom by node
    ndf=6
    ospy.model('basic', '-ndm', 3, '-ndf', ndf)

    # transformações geometricas
    ospy.geomTransf('Linear', 1, 0.0, 0.0, 1.0, '-jntOffset', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    ospy.geomTransf('Linear', 2, 1.0, 0.0, 0.0, '-jntOffset', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [10]:

@nb.jit(["void(int64,float64,int64,float64,int64,float64, float64, float64, float64, float64, float64, float64, float64, float64)"])
def estructSet(vx, dx, vy, dy, vz, dz, pb, pa, pfck, vxb, vxa, vyb, vya, vfck):
    # numero total de pontos
    Npts = (vx*2 +1) * (vy*2 +1) * (vz) + ( (vx+1) * (vy+1) )
    # Matriz dos indices
    mat = np.arange(0,Npts, 1, dtype= np.int64 )
    # corrdenadas dos pontos
    cX = coordX(mat, vx, vy, dx)
    cY = coordY(mat, vx, vy, dy)
    cZ = coordZ(mat, vx, vy, dz)
    # matriz dos andares superiores
    matA = mat[(vX+1)*(vY+1)::].reshape(vZ, vY*2 +1, vX*2 +1)
    # matriz dos indices iniciais das vigas em x
    matXI = matA[::,::2,:-1:]
    # matriz dos indides finais das vigas em x
    matXF = matA[::,::2,1::]
    # matriz dos indices inicias das vigas em Y
    matYI = matA[::,:-1:,::2]
    # matriz dos indices finais das vigas em Y
    matYF = matA[::,1::,::2]
    # matriz pontos da fundação
    matB = mat[: (vx+1) * (vy+1)]
    # matriz final dos pilares
    matPF = matA[::,::2,::2]
    # matriz inicial dos pilares
    matPI = np.append(matB,matPF[:-1:,::,::]).reshape(vZ,(vY+1),(vX+1))
    # Matrizes de superficie 
    matBL = matA[::, :-1:, :-1:]
    #print("Inferior esquerdo\n",matBL)
    matBR = matA[::, :-1:, 1::]
    #print("Inferior direito\n",matBR)
    matTL= matA[::, 1::, :-1:]
    #print("Superior esquerdo\n",matTL)
    matTR = matA[::, 1::, 1::]
    #print("Superior direito\n",matTR)
    
    # reset model
    resetModel()
    # nodes
    ospyNode(mat, cX, cY, cZ)

    # pilares
    startTag = 0
    print("tags dos pilares: ", startTag)
    frameSet(pa, pb, pfck, .2, 2, matPI.flatten(), matPF.flatten(), startTag)

    # vigas x
    startTag = startTag +  matPF.size
    print("Tagas das vigas em X", startTag)
    frameSet(vxa,vxb, vfck, .2, 1, matXI.flatten() , matXF.flatten(), startTag)
    
    #vigas y
    startTag = startTag + matYF.size
    print("Tagas das vigas em Y", startTag)
    frameSet(vya,vyb, vfck, .2, 1, matYI.flatten(), matYF.flatten(), startTag)
    
    #lajes
    


In [11]:
# entradas
## numero de vaos em x
vX = 2
## distancia dos vaos em x - metros
dX = 5.25
## número de vãos em y
vY = 3
## distancia dos vaos em y - - metros
dY = 7.25
## numero pavimentos
vZ = 2
## distancia piso a piso - metros
dZ = 3.25

# pilares
pb = .4 #dimensão x em metros
pa = .5 #dimensão y em metros
pfck = 30 #fck em MPa

# vigas
vxb = .12 # base em metros
vxa = .4 # altura em metros
vyb = .15 # base em metros
vya = .5 # altira em metros
vfck = 30 # Fck em MPa

# lajes


In [12]:
estructSet(vX, dX, vY, dY, vZ, dZ, pb, pa, pfck, vxb, vxa, vyb, vya, vfck)

tags dos pilares:  0
Tagas das vigas em X 24
Tagas das vigas em Y 60


In [13]:
teste = ospy.nodeCoord(40)
print(teste)

[7.875, 18.125, 3.25]


In [14]:
ospy.printModel('-file', 'modelo.txt')